# Install required libraries
!pip install rdkit pandas numpy scikit-learn joblib flask flask-cors pyngrok pymoo deepchem

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Project path (make sure this exists in Drive)
project_dir = "/content/drive/MyDrive/drug-combo-discovery-colab"

# Imports
import pandas as pd
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem


In [ ]:
# Install required libraries
!pip install rdkit pandas numpy scikit-learn joblib flask flask-cors pyngrok pymoo deepchem

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Project path (make sure this exists in Drive)
project_dir = "/content/drive/MyDrive/drug-combo-discovery-colab"

# Imports
import pandas as pd
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 552.4/552.4 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.9/295.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.2 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
import pandas as pd

raw_dir = f"{project_dir}/data/raw"

# Make sure folders exist
import os
os.makedirs(raw_dir, exist_ok=True)

# 1. mock_compounds.csv
mock_compounds = pd.DataFrame({
    "compound_id": ["C1", "C2", "C3", "C4", "C5"],
    "smiles": [
        "CCO",                # ethanol
        "CC(=O)O",            # acetic acid
        "CCN(CC)CC",          # triethylamine
        "CC(C)C(=O)O",        # isobutyric acid
        "CC1=CC=CC=C1"        # ethylbenzene
    ]
})
mock_compounds.to_csv(f"{raw_dir}/mock_compounds.csv", index=False)

# 2. drugcomb_subset.csv
drugcomb_subset = pd.DataFrame({
    "drug_a": ["C1", "C2", "C3", "C4", "C1"],
    "drug_b": ["C2", "C3", "C4", "C5", "C5"],
    "synergy_score": [0.7, 0.4, 0.9, 0.6, 0.8]
})
drugcomb_subset.to_csv(f"{raw_dir}/drugcomb_subset.csv", index=False)

# 3. tox21.csv
tox21 = pd.DataFrame({
    "compound_id": ["C1", "C2", "C3", "C4", "C5"],
    "smiles": [
        "CCO",
        "CC(=O)O",
        "CCN(CC)CC",
        "CC(C)C(=O)O",
        "CC1=CC=CC=C1"
    ],
    "toxic": [0, 1, 0, 1, 0]   # 0 = non-toxic, 1 = toxic
})
tox21.to_csv(f"{raw_dir}/tox21.csv", index=False)

# 4. disgenet_example.csv
disgenet_example = pd.DataFrame({
    "disease": ["lung cancer", "lung cancer", "breast cancer", "breast cancer", "diabetes"],
    "gene": ["TP53", "EGFR", "BRCA1", "BRCA2", "INS"]
})
disgenet_example.to_csv(f"{raw_dir}/disgenet_example.csv", index=False)

print("✅ All starter CSVs created in", raw_dir)



✅ All starter CSVs created in /content/drive/MyDrive/drug-combo-discovery-colab/data/raw


In [ ]:
def smiles_to_morgan(smiles, radius=2, nBits=1024):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=nBits)
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

# Example: add fingerprints to compounds
compound_features = []
for smi in mock_compounds["smiles"]:
    arr = smiles_to_morgan(smi)
    compound_features.append(arr)

X_compounds = np.array(compound_features)
# Make sure processed data folder exists
processed_dir = f"{project_dir}/data/processed"
import os
os.makedirs(processed_dir, exist_ok=True)

np.save(f"{processed_dir}/compound_features.npy", X_compounds)
print("Fingerprints generated:", X_compounds.shape)

[20:36:56] DEPRECATION WARNING: please use MorganGenerator
[20:36:56] DEPRECATION WARNING: please use MorganGenerator
[20:36:56] DEPRECATION WARNING: please use MorganGenerator
[20:36:56] DEPRECATION WARNING: please use MorganGenerator
[20:36:56] DEPRECATION WARNING: please use MorganGenerator


Fingerprints generated: (5, 1024)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Set project directory inside your Drive
project_dir = "/content/drive/MyDrive/drug-combo-discovery-colab"

# Create folder structure
import os
folders = [
    f"{project_dir}/data/processed",
    f"{project_dir}/data/results",
    f"{project_dir}/models",
    f"{project_dir}/notebooks",
    f"{project_dir}/flask_app"
]
for p in folders:
    os.makedirs(p, exist_ok=True)

print("Project folder created at:", project_dir)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Project folder created at: /content/drive/MyDrive/drug-combo-discovery-colab


In [ ]:
import pandas as pd, os

raw_dir = f"{project_dir}/data/raw"

mock_compounds = pd.DataFrame({
    "compound_id": ["C1", "C2", "C3", "C4", "C5"],
    "smiles": [
        "CCO",
        "CC(=O)O",
        "CCN(CC)CC",
        "CC(C)C(=O)O",
        "CC1=CC=CC=C1"
    ]
})
mock_compounds.to_csv(os.path.join(raw_dir, "mock_compounds.csv"), index=False)

drugcomb_subset = pd.DataFrame({
    "drug_a": ["C1", "C2", "C3", "C4", "C1"],
    "drug_b": ["C2", "C3", "C4", "C5", "C5"],
    "synergy_score": [0.7, 0.4, 0.9, 0.6, 0.8]
})
drugcomb_subset.to_csv(os.path.join(raw_dir, "drugcomb_subset.csv"), index=False)

tox21 = pd.DataFrame({
    "compound_id": ["C1", "C2", "C3", "C4", "C5"],
    "smiles": ["CCO","CC(=O)O","CCN(CC)CC","CC(C)C(=O)O","CC1=CC=CC=C1"],
    "toxic": [0,1,0,1,0]
})
tox21.to_csv(os.path.join(raw_dir, "tox21.csv"), index=False)

disgenet_example = pd.DataFrame({
    "disease": ["lung cancer", "lung cancer", "breast cancer", "breast cancer", "diabetes"],
    "gene": ["TP53", "EGFR", "BRCA1", "BRCA2", "INS"]
})
disgenet_example.to_csv(os.path.join(raw_dir, "disgenet_example.csv"), index=False)

print("Starter CSVs written to:", raw_dir)
print(os.listdir(raw_dir))


Starter CSVs written to: /content/drive/MyDrive/drug-combo-discovery-colab/data/raw
['mock_compounds.csv', 'tox21.csv', 'disgenet_example.csv', 'drugcomb_subset.csv']


In [ ]:
import pandas as pd
raw_dir = f"{project_dir}/data/raw"
print("Files:", list(pd.io.common.get_handle.__self__ if False else os.listdir(raw_dir)))
print("mock_compounds:")
print(pd.read_csv(f"{raw_dir}/mock_compounds.csv"))


Files: ['mock_compounds.csv', 'tox21.csv', 'disgenet_example.csv', 'drugcomb_subset.csv']
mock_compounds:
  compound_id        smiles
0          C1           CCO
1          C2       CC(=O)O
2          C3     CCN(CC)CC
3          C4   CC(C)C(=O)O
4          C5  CC1=CC=CC=C1


In [ ]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
import numpy as np
import pandas as pd, os

nBits = 1024
def smiles_to_morgan(smiles, radius=2, nBits=nBits):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=nBits)
    arr = np.zeros((nBits,), dtype=int)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

# Load compounds
comp_df = pd.read_csv(os.path.join(raw_dir, "mock_compounds.csv"))
X = []
ids = []
for _, r in comp_df.iterrows():
    arr = smiles_to_morgan(r.smiles)
    if arr is None:
        continue
    X.append(arr)
    ids.append(r.compound_id)
X = np.array(X)
print("Fingerprint matrix:", X.shape)

# save processed features
proc_dir = f"{project_dir}/data/processed"
np.save(os.path.join(proc_dir, "compound_features.npy"), X)
pd.DataFrame({"compound_id": ids}).to_csv(os.path.join(proc_dir, "compound_ids.csv"), index=False)
print("Saved processed features to", proc_dir)


Fingerprint matrix: (5, 1024)


[20:36:59] DEPRECATION WARNING: please use MorganGenerator
[20:36:59] DEPRECATION WARNING: please use MorganGenerator
[20:36:59] DEPRECATION WARNING: please use MorganGenerator
[20:36:59] DEPRECATION WARNING: please use MorganGenerator
[20:36:59] DEPRECATION WARNING: please use MorganGenerator


Saved processed features to /content/drive/MyDrive/drug-combo-discovery-colab/data/processed


In [ ]:
from sklearn.ensemble import RandomForestRegressor
import joblib, numpy as np, os, pandas as pd

proc_dir = f"{project_dir}/data/processed"
model_dir = f"{project_dir}/models"
results_dir = f"{project_dir}/data/results"

X = np.load(os.path.join(proc_dir, "compound_features.npy"))
# For a mock demo we generate a synthetic potency target (seeded for repeatability)
rng = np.random.RandomState(42)
y = rng.rand(X.shape[0])

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X, y)

joblib.dump(rf, os.path.join(model_dir, "rf_qsar.joblib"))
# Write predictions to results
preds = rf.predict(X)
comp_ids = pd.read_csv(os.path.join(proc_dir, "compound_ids.csv"))["compound_id"]
pd.DataFrame({"compound_id": comp_ids, "predicted_potency": preds}).to_csv(os.path.join(results_dir, "compound_scores.csv"), index=False)
print("QSAR trained and saved. Results:", os.path.join(results_dir, "compound_scores.csv"))


QSAR trained and saved. Results: /content/drive/MyDrive/drug-combo-discovery-colab/data/results/compound_scores.csv


In [ ]:
from sklearn.ensemble import RandomForestRegressor
import joblib, numpy as np, os, pandas as pd

proc_dir = f"{project_dir}/data/processed"
model_dir = f"{project_dir}/models"
results_dir = f"{project_dir}/data/results"

X = np.load(os.path.join(proc_dir, "compound_features.npy"))
# For a mock demo we generate a synthetic potency target (seeded for repeatability)
rng = np.random.RandomState(42)
y = rng.rand(X.shape[0])

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X, y)

joblib.dump(rf, os.path.join(model_dir, "rf_qsar.joblib"))
# Write predictions to results
preds = rf.predict(X)
comp_ids = pd.read_csv(os.path.join(proc_dir, "compound_ids.csv"))["compound_id"]
pd.DataFrame({"compound_id": comp_ids, "predicted_potency": preds}).to_csv(os.path.join(results_dir, "compound_scores.csv"), index=False)
print("QSAR trained and saved. Results:", os.path.join(results_dir, "compound_scores.csv"))


QSAR trained and saved. Results: /content/drive/MyDrive/drug-combo-discovery-colab/data/results/compound_scores.csv


In [ ]:
from sklearn.ensemble import RandomForestClassifier
import joblib, numpy as np, os, pandas as pd

# prepare tox X,y
tox = pd.read_csv(os.path.join(raw_dir, "tox21.csv"))
X_tox = []
y_tox = []
for _, r in tox.iterrows():
    arr = smiles_to_morgan(r.smiles)
    if arr is None: continue
    X_tox.append(arr)
    y_tox.append(int(r.toxic))
X_tox = np.array(X_tox)
y_tox = np.array(y_tox)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_tox, y_tox)
joblib.dump(clf, os.path.join(model_dir, "tox21_clf.joblib"))
print("Toxicity model saved:", os.path.join(model_dir, "tox21_clf.joblib"))


[20:37:06] DEPRECATION WARNING: please use MorganGenerator
[20:37:06] DEPRECATION WARNING: please use MorganGenerator
[20:37:06] DEPRECATION WARNING: please use MorganGenerator
[20:37:06] DEPRECATION WARNING: please use MorganGenerator
[20:37:06] DEPRECATION WARNING: please use MorganGenerator


Toxicity model saved: /content/drive/MyDrive/drug-combo-discovery-colab/models/tox21_clf.joblib


In [ ]:
from sklearn.linear_model import LogisticRegression
import numpy as np, pandas as pd, joblib, os

drugcomb = pd.read_csv(os.path.join(raw_dir, "drugcomb_subset.csv"))
comp_df = pd.read_csv(os.path.join(raw_dir, "mock_compounds.csv")).set_index("compound_id")

pairs_x = []
pairs_y = []
for _, r in drugcomb.iterrows():
    a, b, score = r.drug_a, r.drug_b, r.synergy_score
    smi_a = comp_df.loc[a].smiles
    smi_b = comp_df.loc[b].smiles
    fa = smiles_to_morgan(smi_a)
    fb = smiles_to_morgan(smi_b)
    if fa is None or fb is None: continue
    pairs_x.append(np.concatenate([fa, fb]))
    pairs_y.append(1 if score > 0.5 else 0)

X_pairs = np.array(pairs_x)
y_pairs = np.array(pairs_y)

if len(X_pairs) == 0:
    raise RuntimeError("No valid pairs found in drugcomb_subset.csv")

synergy = LogisticRegression(max_iter=2000)
synergy.fit(X_pairs, y_pairs)
joblib.dump(synergy, os.path.join(model_dir, "synergy_model.joblib"))
print("Synergy model saved:", os.path.join(model_dir, "synergy_model.joblib"))


[20:37:07] DEPRECATION WARNING: please use MorganGenerator
[20:37:07] DEPRECATION WARNING: please use MorganGenerator
[20:37:07] DEPRECATION WARNING: please use MorganGenerator
[20:37:07] DEPRECATION WARNING: please use MorganGenerator
[20:37:07] DEPRECATION WARNING: please use MorganGenerator
[20:37:07] DEPRECATION WARNING: please use MorganGenerator
[20:37:07] DEPRECATION WARNING: please use MorganGenerator
[20:37:07] DEPRECATION WARNING: please use MorganGenerator
[20:37:07] DEPRECATION WARNING: please use MorganGenerator
[20:37:07] DEPRECATION WARNING: please use MorganGenerator


Synergy model saved: /content/drive/MyDrive/drug-combo-discovery-colab/models/synergy_model.joblib


In [ ]:
import itertools, pandas as pd, numpy as np, joblib, os

comp_df = pd.read_csv(os.path.join(raw_dir, "mock_compounds.csv")).set_index("compound_id")
synergy_model = joblib.load(os.path.join(model_dir, "synergy_model.joblib"))

rows = []
comp_ids = list(comp_df.index)
for a,b in itertools.permutations(comp_ids, 2):
    fa = smiles_to_morgan(comp_df.loc[a].smiles)
    fb = smiles_to_morgan(comp_df.loc[b].smiles)
    if fa is None or fb is None: continue
    x = np.concatenate([fa, fb]).reshape(1,-1)
    prob = float(synergy_model.predict_proba(x)[0,1])
    rows.append({"drug_a": a, "drug_b": b, "pred_synergy_prob": prob})

pd.DataFrame(rows).to_csv(os.path.join(results_dir, "pair_predictions.csv"), index=False)
print("Wrote pair_predictions.csv with", len(rows), "rows")


[20:37:08] DEPRECATION WARNING: please use MorganGenerator
[20:37:08] DEPRECATION WARNING: please use MorganGenerator
[20:37:08] DEPRECATION WARNING: please use MorganGenerator
[20:37:08] DEPRECATION WARNING: please use MorganGenerator
[20:37:08] DEPRECATION WARNING: please use MorganGenerator
[20:37:08] DEPRECATION WARNING: please use MorganGenerator
[20:37:08] DEPRECATION WARNING: please use MorganGenerator
[20:37:08] DEPRECATION WARNING: please use MorganGenerator
[20:37:08] DEPRECATION WARNING: please use MorganGenerator
[20:37:08] DEPRECATION WARNING: please use MorganGenerator
[20:37:08] DEPRECATION WARNING: please use MorganGenerator
[20:37:08] DEPRECATION WARNING: please use MorganGenerator
[20:37:08] DEPRECATION WARNING: please use MorganGenerator
[20:37:08] DEPRECATION WARNING: please use MorganGenerator
[20:37:08] DEPRECATION WARNING: please use MorganGenerator
[20:37:08] DEPRECATION WARNING: please use MorganGenerator
[20:37:08] DEPRECATION WARNING: please use MorganGenerat

Wrote pair_predictions.csv with 20 rows


In [ ]:
# Simplified optimizer: optimize propA in [0,1], propB in [0,1]; objective: maximize synergy (from model) - minimize toxicity
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
import numpy as np, pandas as pd, joblib, os

pair_df = pd.read_csv(os.path.join(results_dir, "pair_predictions.csv"))
qsar = joblib.load(os.path.join(model_dir, "rf_qsar.joblib"))
tox_model = joblib.load(os.path.join(model_dir, "tox21_clf.joblib"))

# select top K pairs by predicted synergy prob
top_pairs = pair_df.sort_values("pred_synergy_prob", ascending=False).head(3).reset_index(drop=True)
print("Top pairs:", top_pairs.head())

def get_synergy_for_pair(a,b,propA,propB):
    # mock: use predicted synergy prob * (1 - |propA - propB|)
    row = pair_df[(pair_df.drug_a==a)&(pair_df.drug_b==b)].iloc[0]
    base = row.pred_synergy_prob
    score = base * (1 - abs(propA - propB))
    return score

def get_toxicity_for_pair(a,b,propA,propB):
    # mock: combine single-agent toxicity probabilities weighted by prop
    pa = propA
    pb = propB
    # predict toxicity prob per agent (use tox model)
    fa = smiles_to_morgan(comp_df.loc[a].smiles).reshape(1,-1)
    fb = smiles_to_morgan(comp_df.loc[b].smiles).reshape(1,-1)
    ta = tox_model.predict_proba(fa)[0,1]
    tb = tox_model.predict_proba(fb)[0,1]
    # combined toxicity approx
    return pa*ta + pb*tb

class MixProblem(ElementwiseProblem):
    def __init__(self, a,b):
        super().__init__(n_var=2, n_obj=2, n_constr=0, xl=np.array([0.0,0.0]), xu=np.array([1.0,1.0]))
        self.a = a; self.b = b
    def _evaluate(self, x, out, *args, **kwargs):
        pa, pb = x[0], x[1]
        # we want to maximize synergy and minimize toxicity -> since pymoo minimizes, use -synergy
        synergy = get_synergy_for_pair(self.a, self.b, pa, pb)
        toxicity = get_toxicity_for_pair(self.a, self.b, pa, pb)
        out["F"] = [-synergy, toxicity]

final_results = []
for idx, r in top_pairs.iterrows():
    a = r.drug_a; b = r.drug_b
    prob = r.pred_synergy_prob
    problem = MixProblem(a,b)
    algorithm = NSGA2(pop_size=20)
    res = minimize(problem, algorithm, ('n_gen', 30), verbose=False)
    X = res.X
    F = res.F
    for xi, fi in zip(X, F):
        pa, pb = float(xi[0]), float(xi[1])
        synergy_est = -float(fi[0])
        tox_est = float(fi[1])
        final_results.append({"drug_a":a,"drug_b":b,"propA":pa,"propB":pb,"predicted_synergy":synergy_est,"predicted_toxicity":tox_est})
pd.DataFrame(final_results).to_csv(os.path.join(results_dir, "combo_mixtures.csv"), index=False)
print("Wrote combo_mixtures.csv with", len(final_results), "rows")


Top pairs:   drug_a drug_b  pred_synergy_prob
0     C3     C5           0.963731
1     C1     C5           0.959859
2     C3     C4           0.946882


[20:37:10] DEPRECATION WARNING: please use MorganGenerator
[20:37:10] DEPRECATION WARNING: please use MorganGenerator
[20:37:10] DEPRECATION WARNING: please use MorganGenerator
[20:37:10] DEPRECATION WARNING: please use MorganGenerator
[20:37:10] DEPRECATION WARNING: please use MorganGenerator
[20:37:10] DEPRECATION WARNING: please use MorganGenerator
[20:37:10] DEPRECATION WARNING: please use MorganGenerator
[20:37:10] DEPRECATION WARNING: please use MorganGenerator
[20:37:10] DEPRECATION WARNING: please use MorganGenerator
[20:37:10] DEPRECATION WARNING: please use MorganGenerator
[20:37:10] DEPRECATION WARNING: please use MorganGenerator
[20:37:10] DEPRECATION WARNING: please use MorganGenerator
[20:37:10] DEPRECATION WARNING: please use MorganGenerator
[20:37:10] DEPRECATION WARNING: please use MorganGenerator
[20:37:10] DEPRECATION WARNING: please use MorganGenerator
[20:37:10] DEPRECATION WARNING: please use MorganGenerator
[20:37:10] DEPRECATION WARNING: please use MorganGenerat

Wrote combo_mixtures.csv with 9 rows


In [ ]:
import pandas as pd, os
print(pd.read_csv(os.path.join(results_dir, "combo_mixtures.csv")).head(10))


  drug_a drug_b         propA         propB  predicted_synergy  \
0     C3     C5  1.365286e-08  1.140786e-07           0.963731   
1     C3     C5  1.038889e-06  9.808589e-07           0.963731   
2     C1     C5  4.195415e-06  4.600329e-06           0.959859   
3     C1     C5  3.166792e-06  2.104902e-06           0.959858   
4     C1     C5  6.330101e-07  4.600329e-06           0.959855   
5     C1     C5  6.416566e-08  4.474887e-06           0.959855   
6     C3     C4  3.554489e-08  2.933152e-08           0.946882   
7     C3     C4  3.331843e-08  3.339158e-08           0.946882   
8     C3     C4  2.992929e-08  2.045534e-08           0.946882   

   predicted_toxicity  
0        1.961786e-08  
1        2.608263e-07  
2        1.407319e-06  
3        8.434709e-07  
4        8.373343e-07  
5        7.262484e-07  
6        2.848628e-08  
7        3.171469e-08  
8        2.037996e-08  


In [ ]:
!pip install fastapi uvicorn nest_asyncio pyngrok


In [ ]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import pandas as pd
import os
from pyngrok import ngrok

nest_asyncio.apply()
app = FastAPI()

# Enable CORS for frontend
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Path to your results
RESULTS_PATH = "/content/drive/MyDrive/drug-combo-discovery-colab/data/results/combo_mixtures.csv"

@app.get("/predict")
def predict(disease: str = "diabetes"):
    if not os.path.exists(RESULTS_PATH):
        return {"error": "No results found. Run ML pipeline first."}
    df = pd.read_csv(RESULTS_PATH)
    return df.to_dict(orient="records")

@app.post("/api/search")
def search(data: dict):
    disease_name = data.get("disease_name", "diabetes")
    return predict(disease_name)

@app.post("/api/investigate")
def investigate(data: dict):
    return predict()

@app.get("/api/download/{disease}")
def download(disease: str):
    if not os.path.exists(RESULTS_PATH):
        return {"error": "No results found"}
    df = pd.read_csv(RESULTS_PATH)
    return df.to_dict(orient="records")

# ✅ Ngrok tunnel (reconnect safe)
if "public_url" not in globals():
    ngrok.set_auth_token("32tL4Yx3uFuVIkG90l7gWwWzweO_6PZzoWq9uqhxF8wYAMyRE")  # replace with your actual token
    public_url = ngrok.connect(8000).public_url
    print("🚀 FastAPI running at:", public_url)

uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [5971]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🚀 FastAPI running at: https://cca8fa9700bc.ngrok-free.app
INFO:     103.203.173.18:0 - "OPTIONS /predict?disease=lung%20cancer HTTP/1.1" 200 OK
INFO:     103.203.173.18:0 - "OPTIONS /health HTTP/1.1" 200 OK
INFO:     103.203.173.18:0 - "GET /health HTTP/1.1" 404 Not Found
INFO:     103.203.173.18:0 - "OPTIONS /health HTTP/1.1" 200 OK
INFO:     103.203.173.18:0 - "OPTIONS /health HTTP/1.1" 200 OK
INFO:     103.203.173.18:0 - "GET /health HTTP/1.1" 404 Not Found
INFO:     103.203.173.18:0 - "GET /health HTTP/1.1" 404 Not Found
INFO:     103.203.173.18:0 - "OPTIONS /health HTTP/1.1" 200 OK
INFO:     103.203.173.18:0 - "OPTIONS /health HTTP/1.1" 200 OK
INFO:     103.203.173.18:0 - "GET /health HTTP/1.1" 404 Not Found
INFO:     103.203.173.18:0 - "GET /health HTTP/1.1" 404 Not Found


In [ ]:
!fuser -k 8000/tcp
